In [1]:
import pandas as pd
import datetime as dt
import time
import calendar

from uce_resources import get_site_id, get_forecast, put_forecast

In [2]:
from settings.sites import ceg as sites_list

target_year = 2023
target_month = 2
forecasts_types = ['real', 'naive', 'zero']

# sites_list = ['Balivka']
sites_data = dict.fromkeys(sites_list)
print(sites_data)

{'Bar': None, 'Verkhivka': None, 'Balky': None, 'Sharhorod_1': None, 'Chechelnyk_1': None, 'Chechelnyk_2': None, 'Stanislavchyk': None, 'Kopaihorod': None, 'Cherniatka': None, 'Hlybochok_1': None, 'Hlybochok_2.1': None, 'Hlybochok_2.2': None, 'Pohrebyshche': None, 'Bilashky': None, 'Porohy': None, 'Hnatkiv': None, 'Myroliubivka': None, 'Kyselivka': None, 'Poniativka': None, 'Kostohryzove': None, 'Bilozerka': None, 'Mykolaivka': None, 'Komyshany_1': None, 'Komyshany_2': None, 'Velihen': None, 'Veliton': None, 'Mala_Lepetykha': None, 'Rubanivka': None, 'Oleshky_2': None, 'Oleshky_1': None, 'Liubymivka': None, 'Kachkarivka': None, 'Vasylivka': None, 'Afanasiivka': None, 'Novokondakove': None, 'Bazaltova': None, 'Yelanets_1': None, 'Yelanets_2': None, 'Inhulets_1': None, 'Inhulets_2': None, 'Bereznehuvate': None, 'Teplychna': None, 'Solone': None, 'Stepnohirsk': None, 'Balivka': None, 'Dibrovka': None, 'Kulevcha': None}


In [3]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.pool import NullPool
from settings.db import DO_URL

engine = create_engine(DO_URL, poolclass=NullPool)
metadata = MetaData()
metadata.reflect(bind=engine)

C:\Users\dmytro\AppData\Local\Temp\ipykernel_8900\3375350544.py:7: SAWarning: Did not recognize type 'point' of column 'location'
  metadata.reflect(bind=engine)


In [4]:
# first_date = dt.date(target_year, target_month, 1)
# last_date = dt.date(target_year, target_month, calendar.monthrange(target_year, target_month)[1])
first_date = dt.date(target_year, target_month, 1)
last_date = dt.date(target_year, target_month, 4)

target_dates = pd.date_range(start=first_date, end=last_date, freq='D').to_pydatetime()

In [5]:

with engine.connect() as connection:
    for date in target_dates:    
        for site in sites_data.keys():
            start = time.time()
            print('-'*50)
            print(site)
            site_id, legal_entity = get_site_id(site, connection, 
                                                metadata.tables['sites'],
                                                include_legal_entity_id=True)
            restored_forecast = get_forecast(site_id, [date], 'restored', connection, metadata)
            update_time = dt.datetime.utcnow()

            time_indexes = restored_forecast.index.to_pydatetime()
            data_values = [int(value * 1000) for value in restored_forecast.tolist()]
            hours = range(1, len(data_values) + 1)

            put_forecast(
                site_id,
                "forecast_applied_raw",
                date.date(),
                hours,
                time_indexes,
                data_values,
                update_time,
                connection=connection
            )

            print(f'Raw forecast is writen: {site} | {date}')



--------------------------------------------------
Bar
restored
2023-02-01
SolarGIS_hourly_29_Bar_20230131_20230214_202301312001.csv
2023-02-01 03:10:00
SolarGIS_hourly_29_Bar_20230131_20230214_202301312001.csv
2023-02-01 04:10:00
SolarGIS_hourly_29_Bar_20230201_20230215_202302010401.csv
2023-02-01 05:10:00
SolarGIS_hourly_29_Bar_20230201_20230215_202302010501.csv
2023-02-01 06:10:00
SolarGIS_hourly_29_Bar_20230201_20230215_202302010601.csv
2023-02-01 07:10:00
SolarGIS_hourly_29_Bar_20230201_20230215_202302010701.csv
2023-02-01 08:10:00
SolarGIS_hourly_29_Bar_20230201_20230215_202302010801.csv
2023-02-01 09:10:00
SolarGIS_hourly_29_Bar_20230201_20230215_202302010901.csv
2023-02-01 10:10:00
SolarGIS_hourly_29_Bar_20230201_20230215_202302011001.csv
2023-02-01 11:10:00
SolarGIS_hourly_29_Bar_20230201_20230215_202302011101.csv
2023-02-01 12:10:00
SolarGIS_hourly_29_Bar_20230201_20230215_202302011201.csv
2023-02-01 13:10:00
SolarGIS_hourly_29_Bar_20230201_20230215_202302011301.csv
2023-02-0

AttributeError: 'datetime.date' object has no attribute 'date'